In [3]:
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [4]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [5]:
with open('password.txt') as f:
    password = f.read()

In [6]:
email = 'jhsuslovemy@naver.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [7]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [8]:
# 로그인 정보 - 나중에 저장
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [9]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

## 2. 제주도 맛집으로 검색

In [10]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [11]:
driver.get(url)
time.sleep(5)

## 3. 첫번째 게시글 열기

In [12]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [13]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [14]:
import unicodedata

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'<광고>섭지코지에서 한끼 맛있게 드시고 싶다면...특히 점심메뉴로 강력추천드리고요 물론 저녁에 먹어도 좋아요!⠀혼술 / 혼밥 가능한곳!!⠀🍱#섭지코지한끼🍱오픈 11:00 - 22:00 (마지막오더 21:00)🍱18시 이후 야식포장 배달가능🍱18시 이후 혼밥손님만 홀가능🍱휴일은 매달 첫번째 세번째 월요일 휴무🍱포장배달가능 & 배달의민족 가능🍱문의전화 : 📱 010-5385-0995🍱단체주문환영🍱유치원, 행사도시락 모두주문가능✳️음료만 별도로 테이크아웃 가능합니다.⠀#섭지코지맛집 #섭지코지 #섭지코지카페#스누피가든 #성산일출봉 #제주핫플 #제주맛집#제주도맛집 #제주도돈까스 #비자림맛집#제주데이트코스 #제주혼자여행 #제주도혼자여행#제주혼술하기좋은곳 #제주혼밥 #제주도혼밥#제주술집 #성산돈까스 #제주혼술 #제주도혼술#서귀포혼술 #제주아쿠아플라넷 #아쿠아플라넷제주'

In [15]:
# 문자에서 #을 이용해 해시태그를 분류해내기.. '#[^\s#, \\]+'

import re
tags = re.findall(r'#[^\s#, \\]+', content)
tags

['#섭지코지한끼🍱오픈',
 '#섭지코지맛집',
 '#섭지코지',
 '#섭지코지카페',
 '#스누피가든',
 '#성산일출봉',
 '#제주핫플',
 '#제주맛집',
 '#제주도맛집',
 '#제주도돈까스',
 '#비자림맛집',
 '#제주데이트코스',
 '#제주혼자여행',
 '#제주도혼자여행',
 '#제주혼술하기좋은곳',
 '#제주혼밥',
 '#제주도혼밥',
 '#제주술집',
 '#성산돈까스',
 '#제주혼술',
 '#제주도혼술',
 '#서귀포혼술',
 '#제주아쿠아플라넷',
 '#아쿠아플라넷제주']

In [16]:
# 작성일자 정보 가져오기 _1o9PC Nzb55
date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]
date

'2021-08-18'

In [17]:
# 좋아요 수 가져오기 
# print(soup.select_one('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.EDfFK.ygqzn > div > div > a > span').text)
try:
    # like = soup.select_one('div.Nm9Fw.zV_NJ').text
    # like = soup.select_one('/html/body/div[6]/div[2]/div/article/div[3]/section[2]/div/div/a/span').text
    like = int(soup.select_one('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.EDfFK.ygqzn > div > div > a > span').text.replace(',',''))
except:
    like = 0
like

0

In [18]:
# 위치정보 가져오기
try:
    place = soup.select('div._M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [19]:
row = [content, date, like, place, tags]
row

['<광고>섭지코지에서 한끼 맛있게 드시고 싶다면...특히 점심메뉴로 강력추천드리고요 물론 저녁에 먹어도 좋아요!⠀혼술 / 혼밥 가능한곳!!⠀🍱#섭지코지한끼🍱오픈 11:00 - 22:00 (마지막오더 21:00)🍱18시 이후 야식포장 배달가능🍱18시 이후 혼밥손님만 홀가능🍱휴일은 매달 첫번째 세번째 월요일 휴무🍱포장배달가능 & 배달의민족 가능🍱문의전화 : 📱 010-5385-0995🍱단체주문환영🍱유치원, 행사도시락 모두주문가능✳️음료만 별도로 테이크아웃 가능합니다.⠀#섭지코지맛집 #섭지코지 #섭지코지카페#스누피가든 #성산일출봉 #제주핫플 #제주맛집#제주도맛집 #제주도돈까스 #비자림맛집#제주데이트코스 #제주혼자여행 #제주도혼자여행#제주혼술하기좋은곳 #제주혼밥 #제주도혼밥#제주술집 #성산돈까스 #제주혼술 #제주도혼술#서귀포혼술 #제주아쿠아플라넷 #아쿠아플라넷제주',
 '2021-08-18',
 0,
 '',
 ['#섭지코지한끼🍱오픈',
  '#섭지코지맛집',
  '#섭지코지',
  '#섭지코지카페',
  '#스누피가든',
  '#성산일출봉',
  '#제주핫플',
  '#제주맛집',
  '#제주도맛집',
  '#제주도돈까스',
  '#비자림맛집',
  '#제주데이트코스',
  '#제주혼자여행',
  '#제주도혼자여행',
  '#제주혼술하기좋은곳',
  '#제주혼밥',
  '#제주도혼밥',
  '#제주술집',
  '#성산돈까스',
  '#제주혼술',
  '#제주도혼술',
  '#서귀포혼술',
  '#제주아쿠아플라넷',
  '#아쿠아플라넷제주']]

## 5. 다음 게시글로 이동

In [20]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(1)

In [21]:
# 열린 창 닫기 body > div._2dDPU.CkGkG > div.Igw0E.IwRSH.eGOV_._4EzTm.BI4qX.qJPeX.fm1AK.TxciK.yiMZG > button
# /html/body/div[6]/div[3]/button
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

## 6. 다중으로 크롤링하기

In [22]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [23]:
results = []

for i in range(50):
    print(i, end=' ')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#, \\]+', content)
    date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]

    try:
    # like = soup.select_one('div.Nm9Fw.zV_NJ').text
    # like = soup.select_one('/html/body/div[6]/div[2]/div/article/div[3]/section[2]/div/div/a/span').text
        # like = int(soup.select_one('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.EDfFK.ygqzn > div > div > a > span').text.replace(',',''))
        like = int(soup.select_one('div.Nm9Fw').text[4:-1].replace(',',''))
    except:
        like = 0

    try:
        place = soup.select('div._M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    row = [content, date, like, place, tags]
    results.append(row)

    driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
    time.sleep(2)

0 1 

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

## 500개의 데이터를 가져오기.

## - 제주도 맛집, 제주맛집, 제주도 관광, 제주여행

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
# from tqdm.notebook import tqdm
from urllib.parse import quote
import unicodedata
import re
import time
import pandas as pd

In [25]:
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [27]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#, \\]+', content)
    date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]

    try:
        like = int(soup.select_one('div.Nm9Fw').text[4:-1].replace(',',''))
    except:
        like = 0

    try:
        place = soup.select('div._M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, tags, date, like, place]

In [28]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [ ]:
results = []

for _ in range(500):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

In [ ]:
df.head()
# df.to_csv(f'C:\workspace\02.DataAnalysis\05.Crwaling\data', index=False)

In [30]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(3)

    results = []

    for _ in range(200):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

    df = pd.DataFrame(results, columns=['content', 'tags', 'date', 'like', 'place'])
    df.to_csv(f'./data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(5)

KeyboardInterrupt: 